<a href="https://colab.research.google.com/github/estermartins/Machine-Learning/blob/master/MLtitanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MACHINE LEARNING

Data: 25/03/2019

Professor: Neylson Crepalde

Ester Pereira Martins


**Classificação - Regressão Logística**

Vamos estimar modelos de regressão logística para entender as chances de sobrevivência mo Titanic a partir de atributos disponíveis.

In [0]:
# Importar os módulo necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.discrete.discrete_model import Logit
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [3]:
bd = pd.read_csv("https://raw.githubusercontent.com/rebeccabilbro/titanic/master/data/train.csv")
bd.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
bd.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [5]:
bd["Pclass"] = bd["Pclass"].astype("category")
bd.dtypes

PassengerId       int64
Survived          int64
Pclass         category
Name             object
Sex              object
Age             float64
SibSp             int64
Parch             int64
Ticket           object
Fare            float64
Cabin            object
Embarked         object
dtype: object

In [6]:
# Explicar sobrevivência pela idade, sexo e pela classe em que a pessoa estava viajando 
# Montanod o modelo
# Y = Survived
# X = Age, Sex, Pclass
bd1 = pd.get_dummies(bd, drop_first=True)
bd1.columns

Index(['PassengerId', 'Survived', 'Age', 'SibSp', 'Parch', 'Fare', 'Pclass_2',
       'Pclass_3', 'Name_Abbott, Mr. Rossmore Edward',
       'Name_Abbott, Mrs. Stanton (Rosa Hunt)',
       ...
       'Cabin_F G63', 'Cabin_F G73', 'Cabin_F2', 'Cabin_F33', 'Cabin_F38',
       'Cabin_F4', 'Cabin_G6', 'Cabin_T', 'Embarked_Q', 'Embarked_S'],
      dtype='object', length=1727)

In [16]:
# Separendo as variáveis
bd1 = bd1[["Age", "Sex_male", "Pclass_2", "Pclass_3", "Survived"]]
# Remover os NAs
bd1=bd1.dropna()
bd1.shape

(714, 5)

In [18]:
# Criando as matrizes para a estimação 

y = bd1.Survived
x = bd1[["Age", "Sex_male", "Pclass_2", "Pclass_3"]]
x.shape
y.shape


(714,)

In [19]:
x["intercept"] = 1
x.shape

(714, 5)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y , test_size=.3, random_state=1)

In [23]:
# Estimar o modelo e visualizar os reultados 
model1 = Logit(y_train, x_train).fit()
print(model1.summary2())

Optimization terminated successfully.
         Current function value: 0.462052
         Iterations 6
                        Results: Logit
Model:              Logit            No. Iterations:   6.0000  
Dependent Variable: Survived         Pseudo R-squared: 0.320   
Date:               2019-03-25 23:19 AIC:              471.1283
No. Observations:   499              BIC:              492.1913
Df Model:           4                Log-Likelihood:   -230.56 
Df Residuals:       494              LL-Null:          -339.28 
Converged:          1.0000           Scale:            1.0000  
----------------------------------------------------------------
            Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
----------------------------------------------------------------
Age        -0.0316    0.0090   -3.5110  0.0004  -0.0492  -0.0139
Sex_male   -2.5629    0.2455  -10.4413  0.0000  -3.0440  -2.0818
Pclass_2   -1.2833    0.3271   -3.9229  0.0001  -1.9245  -0.6422
Pclass_3   -2.3819   

In [24]:
# Calcular as chances relativas
np.exp(model1.params)

Age           0.968918
Sex_male      0.077081
Pclass_2      0.277114
Pclass_3      0.092376
intercept    36.844502
dtype: float64

In [0]:
def get_oddsperc(x): 
  res = (np.exp(x) - 1) * 100
  return (res)

In [26]:
get_oddsperc(model1.params)

Age            -3.108190
Sex_male      -92.291929
Pclass_2      -72.288630
Pclass_3      -90.762383
intercept    3584.450154
dtype: float64

In [31]:
# Predizendo a minha chance de sobrevivência
Ester = pd.DataFrame({"Age":19, "Sex_male":0, "Pclass_2": 0, "Pclass_3":1, "intercept":1}, index=[0])
print(model1.predict(Ester))

0    0.848573
dtype: float64
